In [1]:
import re
import sys
import string
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

## NOTE: PASS IN WRITING SAMPLE ALREADY PREPROCESSED LIKE BELOW, BUT DEFINE MARKS_TO_WRDS IN THIS NOTEBOOK

In [2]:
# Test comment string, can be deleted.
comment_list = ["35, I have a pretty good job but I don't love it. I'm still trying to figure out what I want to be when I grow up. Situation could be a lot worse but I'm at a point where I want to do something I enjoy.", "My family (live with my real mom and step dad) act like me being outed by older sister never happened. And on top of all of this I'm struggling trying to study for my math ged which I failed six times with being 1 point shy from passing. If I fail it again, its another two months of waiting to retake it.", "I'm really not sure how i'm supposed to feel anymore knowing that most of my life I was just wasting my time.", "I'm an IT apprentice and was about a month away from finishing my level 4 course. My training provider has just closed down due to a fraud investigation so now im in a position of trying to find a provider for my specific course so I can finish. And there's no guarantee my years work can be transferred so I may not even be able to get my certificate without another years of work.", "29 and I don't like being a mum", "I was going to complain about being a 24yo male who's single and doesn't want to be but some of these responses make mine a drop in the bucket compared to the waters some of you face. Best of luck to all of you.", "Split up with my long term so of 5 years and decided to go for a complete career change. Going to pack in this life and go travel for a bit. Experience the world and figure out what it is I want.", "I'm 30, diagnosed last year with ADHS, failed university this year, just needed to write my thesis, no intimacy with my girlfriend of 2 years anymore. Yeah Life!", "32, alcoholic, narcissistic, sociopath, broke, in debt, and i feel eventually my finances will magically fix themselves. I’m also a cocky piece of shit who doesn’t take advice or think there’s anything wrong with me. I truthfully think I’m ok. I need help. So much.", "My dad has lung cancer and is refusing treatment, which is fine. He's a 68 year old man who loves to drink and smoke and has accepted the hand he was dealt. He doesn't have to go through treatment if he doesn't want too."]

In [3]:
'''Test comment preprocessing, can be deleted after integration?'''

# This cell deals with reformatting of the comment-array input.

# Concatinates all the comments into one sample of the Redditor's writing (string object).
writing_sample = ' @ '.join(comment_list)

# Now, we need to check for instances of elipses and assign them to a new symbol--less confusion with periods, easier analysis.
# The tilde will represent an elipse from here on out.
writing_sample = writing_sample.replace('...', '~')


# We will loop over every character in the string and extract the punctuation, appending each mark to a list.
# We will keep this punctuation sequence in both list and string form for further analysis.
marks = ['.',',',':',';','?','!','-','(',')','"',"'",'~']
marks_str = ''.join(map(str, marks))

'''NEW (just the line below this)'''
marks_to_wrds = {' prd ': '.', ' cmma ': ',', ' coln ': ':', ' smicln ': ';', ' qstn ': '?', ' xclm ': '!', ' dsh ': '-', ' lftparen ': '(', ' rghtparen ': ')', ' dblqt ': '"', ' snglqt ': "'", ' elpss ': '~'}

punct_list = []

for character in writing_sample:
    if character in marks:
        punct_list.append(character)
    
punct_str = ''.join(punct_list)

In [ ]:
def get_fp(writing_sample):
    
    transactions = []
    fqt_pat_sample = writing_sample

    for character in fqt_pat_sample:
        if character in marks:
            fqt_pat_sample = fqt_pat_sample.replace(character, list(marks_to_wrds.keys())[list(marks_to_wrds.values()).index(character)])

    wrd_pos = 0
    for word in fqt_pat_sample.split():
        grab_three = []
        word_formatted = ' ' + word + ' '
        if word_formatted in marks_to_wrds.keys():
            if wrd_pos >= 3:
                grab_three.append(fqt_pat_sample.split()[wrd_pos - 3])
                grab_three.append(fqt_pat_sample.split()[wrd_pos - 2])
                grab_three.append(fqt_pat_sample.split()[wrd_pos - 1])
                grab_three.append(fqt_pat_sample.split()[wrd_pos])
            
                transactions.append(grab_three) 
        wrd_pos += 1
    
    # Put the transactions into dataframe form, this allows us to perform FPgrowth on the resultant dataframe.
    te = TransactionEncoder()
    te_transformed = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(te_transformed, columns = te.columns_)

    # Apply FPgrowth to the transactional dataframe.
    freq_pats = fpgrowth(df, min_support = 0.5, use_colnames = True)
    return freq_pats